In [1]:
import numpy as np
import pandas as pd
import h5py
import scipy
import scanpy as sc
import anndata
import pkg_resources
from scbasset.utils import *

# plotting functions
import seaborn as sns
import matplotlib.pyplot as plt

import os

### read example from 10x multiome output

In [2]:
data_path = '../../data/multiome_example/'

h5_file = data_path + '/raw_multiome/pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5'
bed_file = data_path + '/raw_multiome/pbmc_granulocyte_sorted_3k_atac_peaks.bed'

In [3]:
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'])
ad = sc.read_10x_h5(h5_file, gex_only=False)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


### filtering

In [4]:
ad_rna = ad[:, ad.var['feature_types']=='Gene Expression']
ad_atac = ad[:, ad.var['feature_types']=='Peaks']
ad_atac.var['chr'] = peak['chr'].values
ad_atac.var['start'] = peak['start'].values
ad_atac.var['end'] = peak['end'].values

# basic stats
sc.pp.filter_cells(ad_rna, min_genes=0)
sc.pp.filter_genes(ad_rna, min_cells=0)
sc.pp.filter_cells(ad_atac, min_genes=0)
sc.pp.filter_genes(ad_atac, min_cells=0)

# a gene need to be expressed in 5% cells
# a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad_rna = ad_rna[:, ad_rna.var['n_cells']>thres]
ad_atac = ad_atac[:, ad_atac.var['n_cells']>thres]

Trying to set attribute `.var` of view, copying.
Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


### save h5ad and filtered bed

In [5]:
os.makedirs('%s/processed'%data_path, exist_ok=True)

chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad_atac = ad_atac[:, ad_atac.var['chr'].isin(chrs)]
ad_atac.write('%s/processed/pbmc_multiome_ad.h5ad'%data_path)
ad_atac.var.loc[:,['chr','start','end']].to_csv('%s/processed/pbmc_multiome_peaks.bed'%data_path, header=False, sep='\t', index=False)

/home/yuanh/.local/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/home/yuanh/.local/lib/python3.7/site-packages/anndata/_core/anndata.py:1237: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.var` of view, copying.
... storing 'feature_types' as categorical
/home/yuanh/.local/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
Trying to set attribute `.

### create inputs for scBasset

In [6]:
filtered_bed_file = '%s/processed/pbmc_multiome_peaks.bed'%data_path # path to save the filtered peak atlas
ad_file = '%s/processed/pbmc_multiome_ad.h5ad'%data_path
output_file ='%s/processed/pbmc_multiome_train_test_val.h5'%data_path

In [8]:
# fasta file depending the the genome build for dataset
# for hg38: please download hg38.fa.gz at: https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/
# unzip and use fasta_file to indicate the path to hg38.fa
fasta_file = '/home/yuanh/genomes/hg38/hg38.fa' 

In [9]:
make_h5(input_ad=ad_file, 
        input_bed=filtered_bed_file, 
        input_fasta=fasta_file, 
        out_file=output_file)

### alternatively you can download my pre-processed files from google cloud bucket

In [ ]:
import subprocess
import os

download_savepath = '../../data/download'
os.makedirs(download_savepath, exist_ok=True)

if not os.path.exists('%s/pbmc_multiome_ad.h5ad'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_multiome_ad.h5ad'%download_savepath, shell=True)

if not os.path.exists('%s/pbmc_multiome_peaks.bed'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_multiome_peaks.bed'%download_savepath, shell=True)

if not os.path.exists('%s/pbmc_multiome_train_test_val.h5'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_multiome_train_test_val.h5'%download_savepath, shell=True)
